# gloal issues

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# load data

In [6]:
from sklearn.datasets import fetch_20newsgroups

news = fetch_20newsgroups(subset='all')
print(type(news.data), type(news.target), type(news.target_names))
print(len(news.data),len(news.target))
print(news.target_names)
print(news.data[0])
print(news.target[0], news.target_names[news.target[0]])

<class 'list'> <class 'numpy.ndarray'> <class 'list'>
18846 18846
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jag

# Split data

In [7]:
SPLIT_PERC = 0.75
split_size = int(len(news.data)*SPLIT_PERC)
X_train = news.data[:split_size]
X_test = news.data[split_size:]
y_train = news.target[:split_size]
y_test = news.target[split_size:]


# Pipelines with differents vectorizer

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

clf_1 = Pipeline([
   ('vect', CountVectorizer()),
   ('clf', MultinomialNB()),
])

clf_2 = Pipeline([
   ('vect', HashingVectorizer(non_negative=True)),
   ('clf', MultinomialNB()),
])

clf_3 = Pipeline([
   ('vect', TfidfVectorizer()),
   ('clf', MultinomialNB()),
])

# k-fold validation

In [10]:
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(clf, X, y, K):
    # create a k-fold croos validation iterator of k=5 folds
    cv = KFold(len(y), K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(np.mean(scores), sem(scores)))
    
clfs = [clf_1, clf_2, clf_3]
for clf in clfs:
    evaluate_cross_validation(clf, news.data, news.target, 5)
    

[ 0.85782493  0.85725657  0.84664367  0.85911382  0.8458477 ]
Mean score: 0.853 (+/-0.003)
[ 0.75543767  0.77659857  0.77049615  0.78508888  0.76200584]
Mean score: 0.770 (+/-0.005)
[ 0.84482759  0.85990979  0.84558238  0.85990979  0.84213319]
Mean score: 0.850 (+/-0.004)


# Improve vectorizer

In [13]:
clf_4 = Pipeline([
    ('vect', TfidfVectorizer(
        token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
        )),
    ('clf', MultinomialNB()),
])

evaluate_cross_validation(clf_4, news.data, news.target, 5)

[ 0.86100796  0.8718493   0.86203237  0.87291059  0.8588485 ]
Mean score: 0.865 (+/-0.003)


# Improve classifier

In [14]:
clf_5 = Pipeline([
    ('vect', TfidfVectorizer(
        token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b",
        )),
    ('clf', MultinomialNB(alpha=0.01)),
])

evaluate_cross_validation(clf_5, news.data, news.target, 5)

[ 0.9198939   0.919342    0.91748474  0.92491377  0.91775007]
Mean score: 0.920 (+/-0.001)


# Performance

In [15]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print("Accuracy on training set:")
    print( clf.score(X_train, y_train))
    print( "Accuracy on testing set:")
    print( clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print( "Classification Report:")
    print( metrics.classification_report(y_test, y_pred))
    print( "Confusion Matrix:")
    print( metrics.confusion_matrix(y_test, y_pred))
    
train_and_evaluate(clf_5, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.996674685156
Accuracy on testing set:
0.915959252971
Classification Report:
             precision    recall  f1-score   support

          0       0.95      0.88      0.91       216
          1       0.83      0.84      0.83       246
          2       0.91      0.83      0.87       274
          3       0.81      0.86      0.83       235
          4       0.87      0.90      0.89       231
          5       0.89      0.91      0.90       225
          6       0.88      0.80      0.84       248
          7       0.93      0.93      0.93       275
          8       0.96      0.98      0.97       226
          9       0.97      0.94      0.96       250
         10       0.97      1.00      0.98       257
         11       0.96      0.98      0.97       261
         12       0.90      0.91      0.91       216
         13       0.94      0.96      0.95       257
         14       0.94      0.96      0.95       246
         15       0.90      0.97      0.93     

In [18]:
print( len(clf_5.named_steps['vect'].get_feature_names()))

146047
